In [2]:
import numpy as np
import scipy.special

In [3]:
# neural network class definition
class neuralNetwork:
    
    # initialise the neural network
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        # set number of nodes in each input layer, hidden layer, output layer
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes
    
        #link weight matrices, wih and who are weight matrices
        #weights inside the arrays are w_i_j, where link is from node i to node j in the next layer
        # aka [w11 w21]
        #     [w12 w22]
        self.wih = np.random.normal(0.0, pow(self.hnodes, -0.5), (self.hnodes, self.inodes))
        self.who = np.random.normal(0.0, pow(self.onodes, -0.5), (self.onodes, self.hnodes))
        
        #learning rate
        self.lr = learningrate
        
        # activiation function is the sigmoid function
        self.activation_function = lambda x: scipy.special.expit(x)
        
        pass

    # train the neural network
    def train(self, inputs_list, targets_list):
        # convert inputlist to 2d array
        inputs = np.array(inputs_list, ndmin = 2).T
        targets = np.array(targets_list, ndmin = 2).T
        
        # calculate signals into hidden layer
        hidden_inputs = np.dot(self.wih, inputs)
        # calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # calculate signals into final output layer
        final_inputs = np.dot(self.who, hidden_outputs)
        # calculate the signals emerging from final outpu layer
        final_outputs = self.activation_function(final_inputs)
        
        # calculate output layer's error: (target - actual)
        output_errors = targets - final_outputs
        
        # calculate hidden layer's error: hidden layer error is the output_errors, split by weights, recombined at hidden nodes
        hidden_errors = np.dot(self.who.T, output_errors)
        
        # update the weights for the links between the hidden and output layers
        self.who += self.lr * np.dot((output_errors * final_outputs * (1.0 - final_outputs)), np.transpose(hidden_outputs))
        
        # update the weights for the links between the input and hidden layers
        self.wih += self.lr * np.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), np.transpose(inputs))
        pass
    
    # query the neural network
    def query(self, inputs_list):
        # convert inputs list to 2d array
        inputs = np.array(inputs_list, ndmin = 2).T
        
        # calculate signals into hidden layer
        hidden_inputs = np.dot(self.wih, inputs)
        # calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # calculate signals into final output layer
        final_inputs = np.dot(self.who, hidden_outputs)
        # calculate the signals emerging from final outpu layer
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs

In [4]:
# this section use to I/O the traning set and testing set
# (example) coverage vector matrix
training_data_list = np.array([
    [1, 1, 1, 1, 0, 1, 0, 0, 1],
    [1, 0, 0, 0, 1, 1, 1, 1, 0],
    [0, 0, 0, 0, 0, 1, 1, 0, 0],
    [1, 1, 0, 0, 1, 0, 1, 1, 1],
    [1, 1, 1, 0, 1, 1, 1, 1, 1],
    [0, 0, 1, 0, 0, 1, 1, 1, 0],
    [1, 1, 1, 1, 0, 1, 0, 1, 1],
])

# (example) execution result of each coverage vector
training_target_list = np.array([[0, 0, 0, 0, 0, 1, 1]]).T

# (example) virtual coverage vector matrix(testing set)
testing_data_list = np.zeros((len(training_data_list[0]), len(training_data_list[0])))
for i in range(len(testing_data_list[0])):
    testing_data_list[i][i] = 1

In [5]:
# suppose 'N' represent the number of coverage vector (input or sample)
# N = 7 (base ib above coverage data)
N = len(training_data_list)

# suppose 'M' represent the number of statement  
# M = 9 (base on above coverage data)
M = len(training_data_list[0])

# number of input layer, hidden layer, and output layer nodes
input_nodes = M 
hidden_nodes = 3
output_nodes = 1
learning_rate = 0.5
samplesize = N

# create an object of neuralNetwork class
n = neuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)

In [6]:
# training the neural network
iteration = 10000
for i in range(iteration):
    for record in range(len(training_data_list)):
        inputs = np.asfarray(training_data_list[record])
        targets = np.asfarray(training_target_list[record])
        if int(targets[0]) == int(1):
            targets[0] = 0.99
        elif int(targets[0]) == int(0):
            targets[0] = 0.01
        n.train(inputs, targets)
        pass

In [22]:
# testing the neural network
l = np.asfarray(np.arange(len(testing_data_list)))
for record in range(len(testing_data_list)):
    inputs = np.asfarray(testing_data_list[record])
    outputs = n.query(inputs)
    l[record] = outputs
    print("The suspiciousness of 'S%s' is %s" %(record+1, outputs))
    pass

# sorting the statement based on suspiciousness


The suspiciousness of 'S1' is [[0.01217341]]
The suspiciousness of 'S2' is [[0.05874229]]
The suspiciousness of 'S3' is [[0.97116203]]
The suspiciousness of 'S4' is [[0.7991281]]
The suspiciousness of 'S5' is [[0.00907318]]
The suspiciousness of 'S6' is [[0.08289293]]
The suspiciousness of 'S7' is [[0.01269964]]
The suspiciousness of 'S8' is [[0.98931622]]
The suspiciousness of 'S9' is [[0.11312806]]
[0.00907318 0.01217341 0.01269964 0.05874229 0.08289293 0.11312806
 0.7991281  0.97116203 0.98931622]
